In [1]:
import pandas as pd
import numpy as np 
import pickle
import xgboost as xgb
import time
import graphviz
import matplotlib as plt

In [2]:
START_NOTEBOOK = time.time()
SUB=False

In [3]:
pik = pickle.load(open('../saved_datasets/19-3-19,pipeline_A,dateback=12/xtrain_xtest_ytrain_ytest.pickle','rb'))
#x_train , x_test , y_train , y_test = [[x[0]] for x in pik]

KeyboardInterrupt: 

In [ ]:
x_train , x_test , y_train , y_test = pik

In [ ]:
x_train[0].head()

# Preprocessing

In [ ]:
#save x_train[1] to create submission later (becuase i delete shop and item ids)
x_test_1_saved = x_test[1][['shop_id','item_id']]

In [ ]:
def preproc_model3(x_train,x_test,y_train,y_test):
    x_train = [d.drop(['date_block_num','shop_id','item_id','item_category_id','month','city','item_type','item_info'],axis=1) for d in x_train]
    x_test =  [d.drop(['date_block_num','shop_id','item_id','item_category_id','month','city','item_type','item_info'],axis=1) for d in x_test]
    y_train = [d.clip(0,20) for d in y_train]
    y_test =  [d.clip(0,20) for d in y_test]
    return [x_train,x_test,y_train,y_test]

In [ ]:
x_train , x_test , y_train , y_test = preproc_model3(x_train,x_test,y_train,y_test)

In [ ]:
a = x_test[0].columns.tolist()
b = x_train[0].columns.tolist()
a==b

# HACKS

### sort columns

In [ ]:
train_cols = x_train[0].columns.tolist()
x_test[0] = x_test[0][train_cols]

In [ ]:
A = x_train[0].columns.tolist()

In [ ]:
B = x_test[0].columns.tolist()

In [ ]:
A==B

### fillnas in y_test

In [ ]:
#y_test = [d.fillna(0) for d in y_test]

## TODO: account for factor adjustment

# Eval Metric

In [ ]:
np.array([1,2,3]).clip(0,1)

In [ ]:
def model3_eval(preds,true_y):
    preds_clipped = preds.clip(0,20)
    true_y_clipped = true_y.clip(0,20)
    sq_res = (preds_clipped - true_y_clipped)**2
    return  np.sqrt(np.mean(sq_res))

# Objective metric

In [ ]:
#def model3_objective(preds,true_y):
    

In [ ]:
np.mean(y_test[0].isna())

# Notes

- There are likely too many feats, this would explain immediate overfitting
- rmse withou clipped data is gonna put a lot of focus on large outliers
* consider mean abs error

# Feature Selection

In [ ]:
#x_train[0].columns.tolist()

In [ ]:
def feat_selection(x_train,x_test,y_train,y_test):
    FEATS= [
 'sum_item_sales_back_1',
 'sum_shop_sales_back_1',
 'item_cnt_month_back_1',
 'sum_item_cat_sales_back_1',
 'sum_item_cat_shop_sales_back_1',
 'sum_item_sales_back_2',
 'sum_shop_sales_back_2',
 'item_cnt_month_back_2',
 'sum_item_cat_sales_back_2',
 'sum_item_cat_shop_sales_back_2',
 'sum_item_sales_back_3',
 'sum_shop_sales_back_3',
 'item_cnt_month_back_3',
 'sum_item_cat_sales_back_3',
 'sum_item_cat_shop_sales_back_3',
 'sum_item_sales_back_4',
 'sum_shop_sales_back_4',
 'item_cnt_month_back_4',
 'sum_item_cat_sales_back_4',
 'sum_item_cat_shop_sales_back_4',
 'sum_item_sales_back_5',
 'sum_shop_sales_back_5',
 'item_cnt_month_back_5',
 'sum_item_cat_sales_back_5',
 'sum_item_cat_shop_sales_back_5',
 'sum_item_sales_back_6',
 'sum_shop_sales_back_6',
 'item_cnt_month_back_6',
 'sum_item_cat_sales_back_6',
 'sum_item_cat_shop_sales_back_6']#,
# 'sum_item_sales_back_7',
# 'sum_shop_sales_back_7',
# 'item_cnt_month_back_7',
# 'sum_item_cat_sales_back_7',
# 'sum_item_cat_shop_sales_back_7',
# 'sum_item_sales_back_8',
# 'sum_shop_sales_back_8',
# 'item_cnt_month_back_8',
# 'sum_item_cat_sales_back_8',
# 'sum_item_cat_shop_sales_back_8',
# 'sum_item_sales_back_9',
# 'sum_shop_sales_back_9',
# 'item_cnt_month_back_9',
# 'sum_item_cat_sales_back_9',
# 'sum_item_cat_shop_sales_back_9',
# 'sum_item_sales_back_10',
# 'sum_shop_sales_back_10',
# 'item_cnt_month_back_10',
# 'sum_item_cat_sales_back_10',
# 'sum_item_cat_shop_sales_back_10',
# 'sum_item_sales_back_11',
# 'sum_shop_sales_back_11',
# 'item_cnt_month_back_11',
# 'sum_item_cat_sales_back_11',
# 'sum_item_cat_shop_sales_back_11',
# 'sum_item_sales_back_12',
# 'sum_shop_sales_back_12',
# 'item_cnt_month_back_12',
# 'sum_item_cat_sales_back_12',
# 'sum_item_cat_shop_sales_back_12',
# 'numdays',
# 'mean_prevmonth_item_price',
# 'std_prevmonth_item_price',
# 'median_prevmonth_item_price',
# 'median_prevmonth_shop_item_price',
# 'prop_median_item_price']#,
# 'first_sale_date_block',
# 'new_item']#,
            
# dont worry about these encodings, theyre prolly almost redundant
            
# 'item_info_encoded_alpha_0',
# 'shop_id_encoded_alpha_0',
# 'item_id_encoded_alpha_0']#,
# 'item_category_id_encoded_alpha_0',
# 'month_encoded_alpha_0',
# 'city_encoded_alpha_0',
# 'item_type_encoded_alpha_0']#,
# 'item_info_encoded_alpha_50',
# 'shop_id_encoded_alpha_50',
# 'item_id_encoded_alpha_50',
# 'item_category_id_encoded_alpha_50',
# 'month_encoded_alpha_50',
# 'city_encoded_alpha_50',
# 'item_type_encoded_alpha_50',
# 'item_info_encoded_alpha_100',
# 'shop_id_encoded_alpha_100',
# 'item_id_encoded_alpha_100',
# 'item_category_id_encoded_alpha_100',
# 'month_encoded_alpha_100',
# 'city_encoded_alpha_100',
# 'item_type_encoded_alpha_100'
# 'item_info_encoded_alpha_200',
# 'shop_id_encoded_alpha_200',
# 'item_id_encoded_alpha_200',
# 'item_category_id_encoded_alpha_200',
# 'month_encoded_alpha_200',
# 'city_encoded_alpha_200',
# 'item_type_encoded_alpha_200']#,
# 'item_info_encoded_alpha_500',
# 'shop_id_encoded_alpha_500',
# 'item_id_encoded_alpha_500',
# 'item_category_id_encoded_alpha_500',
# 'month_encoded_alpha_500',
# 'city_encoded_alpha_500',
# 'item_type_encoded_alpha_500']#,
# 'item_info_encoded_alpha_1000',
# 'shop_id_encoded_alpha_1000',
# 'item_id_encoded_alpha_1000',
# 'item_category_id_encoded_alpha_1000',
# 'month_encoded_alpha_1000',
# 'city_encoded_alpha_1000',
# 'item_type_encoded_alpha_1000']#,
# 'item_info_encoded_alpha_2000',
# 'shop_id_encoded_alpha_2000',
# 'item_id_encoded_alpha_2000',
# 'item_category_id_encoded_alpha_2000',
# 'month_encoded_alpha_2000',
# 'city_encoded_alpha_2000',
# 'item_type_encoded_alpha_2000']
    
    x_train = [d[FEATS] for d in x_train]
    x_test =  [d[FEATS] for d in x_test]
    return [x_train,x_test,y_train,y_test]

In [ ]:
x_train , x_test , y_train , y_test = feat_selection(x_train,x_test,y_train,y_test)

# Reduce Dataset size

In [ ]:
#x_train_small = [d.sample(frac=1,random_state=0) for d in x_train]
#y_train_small = [d.sample(frac=1,random_state=0) for d in y_train]
#x_test_small = [d.sample(frac=1,random_state=0) for d in x_test]
#y_test_small = [d.sample(frac=1,random_state=0) for d in y_test]

In [23]:
x_train[0].shape

(6186922, 30)

# Fitting

In [24]:
y_exp = pickle.load(open('../gen_data/saved_y_train_list[0].pickle','rb'))
y_exp.shape

(1940455,)

In [ ]:
model = xgb.XGBRegressor(
    seed=0,
    max_depth=3,
    learning_rate=0.5,
    n_estimators=1000,
    objective='reg:linear',
    nthread=8,
    min_child_weight=100
)
eval_set = [(x_train[0],y_train[0]),(x_test[0],y_test[0])]

In [25]:
y_train[0] = y_exp

In [ ]:
model.fit(
    verbose=True,
    X=x_train[0],
    y=y_train[0],
    eval_set=eval_set,
    early_stopping_rounds=10
)

In [ ]:
#np.sqrt(np.mean((model.predict(x_test[0]) - y_test[0])**2))

In [ ]:
#import os
#os.environ["PATH"] += os.pathsep + 'C://ProgramData/Anaconda3/pkgs/graphviz-2.38.0-h6538335_1011/Library/bin'

#os.environ['PATH']

#xgb.plot_tree(model)

In [ ]:
#feat_imp = model.feature_importances_
#feats = x_train[0].columns
#df = pd.DataFrame({'feature':feats,'importance':feat_imp})
#df = df.sort_values('importance',ascending=False)
#print(df.plot(x='feature',y='importance',kind='bar',))
ax = xgb.plot_importance(model,height=0.8,)
ax.figure.set_size_inches(5,10)

# EDA

In [ ]:
item_cat = x_train[0].item_id_encoded_alpha_0
shop_cat = x_train[0].shop_id_encoded_alpha_0
y = y_train[0]

In [ ]:
d = x_train[0]
d['target'] = y_train[0]
print(d.plot(x='item_id_encoded_alpha_0',y='target',kind='scatter'))

In [ ]:
#SUB=True

# Retrain model with all data

# Write Submission

In [ ]:
#retrain on all data
#
#
#
#
#
if SUB:
    preds = model.predict(x_test[1])

In [ ]:
if SUB:
    preds

In [ ]:
if SUB:
    #sample_sub = pd.read_csv('../original_data/sample_submission.csv.gz')
    test0 = pd.read_csv('../original_data/test.csv.gz')

In [ ]:
if SUB:
    submission = x_test_1_saved.copy()
    submission['item_cnt_month'] = preds
    submission = test0.merge(submission).drop(['shop_id','item_id'],axis=1)

In [ ]:
if SUB:
    submission.to_csv('../gen_data/model3_basic_sub.csv',index=False)
    submission.head()

In [ ]:
if SUB:
    submission_adjusted = submission
    submission_adjusted['item_cnt_month'] = 0.2839853/submission_adjusted['item_cnt_month'].mean()*submission_adjusted['item_cnt_month']
    submission.to_csv('../gen_data/model3_basic_sub_adjusted.csv',index=False)
    submission_adjusted.head()